<div style="display: flex; justify-content: space-between; align-items: center; padding: 8px 16px; background: #F8F9FA; border-bottom: 2px solid #E0E0E0; margin: 0; line-height: 1;">
    <div style="font-size: 14px; color: #666;">
        <span style="font-weight: bold; color: #333;">{SOURCE_PLATFORM} → Databricks Migration</span>
        <span style="margin-left: 8px; color: #999;">|</span>
        <span style="margin-left: 8px;">03 - Execute</span>
    </div>
    <div style="display: flex; align-items: center; gap: 8px;">
        <img src="https://cdn.simpleicons.org/snowflake/29B5E8" width="24" height="24"/>
        <span style="color: #999; font-size: 16px;">→</span>
        <img src="https://cdn.simpleicons.org/databricks/FF3621" width="24" height="24"/>
    </div>
</div>


<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>

# Schema and DDL Conversion

## Overview

Before migrating data, you must create the target schema in Databricks. This lesson covers DDL conversion, data type mapping, and best practices for schema design in the Lakehouse. Proper schema design is foundational to successful migration.

## Learning Objectives

By the end of this lesson, you will be able to:
- Understand the schema migration process and key decisions
- Convert DDL from {SOURCE_PLATFORM} to Databricks SQL
- Map source data types to Delta Lake equivalents
- Apply Lakehouse schema best practices (partitioning, clustering, table properties)
- Use automation tools for schema migration

## Target Design

The target state for migrated data is **managed tables in Unity Catalog** using **Delta Lake** (preferred) or **Iceberg** format. This provides full governance, lineage tracking, and optimization capabilities (including *Predictive Optimization*).

### Medallion Architecture

The medallion architecture organizes data into three tiers based on quality and refinement level.

<br />
<div class="mermaid">
flowchart LR
    LAND["<b>Landing Zone</b><br/><i>Raw files, CDC streams,<br/>external sources</i>"]
    subgraph LAKEHOUSE["Unity Catalog Lakehouse"]
        BRONZE["<b>Bronze</b><br/><i>Raw ingestion,<br/>source-aligned</i>"]
        SILVER["<b>Silver</b><br/><i>Cleansed, conformed,<br/>deduplicated</i>"]
        GOLD["<b>Gold</b><br/><i>Business aggregates,<br/>application-ready</i>"]
    end
    CONSUMERS["<b>Consumers</b><br/><i>BI tools, applications,<br/>ML models, APIs</i>"]
    LAND --> BRONZE --> SILVER --> GOLD --> CONSUMERS
    style LAND fill:#e3f2fd,stroke:#1976d2
    style BRONZE fill:#cd7f32,stroke:#8b4513,color:#fff
    style SILVER fill:#c0c0c0,stroke:#808080,color:#000
    style GOLD fill:#ffd700,stroke:#b8860b,color:#000
    style CONSUMERS fill:#e8f5e9,stroke:#4caf50
    style LAKEHOUSE fill:#fff,stroke:#FF3621,stroke-width:2px
</div>
<script type="module"> import mermaid from "https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs"; mermaid.initialize({ startOnLoad: true, theme: "default" }); </script>

### Organizing Medallion Tiers

The medallion tiers can be implemented using **catalogs** or **schemas** depending on your governance and isolation requirements.

| Approach | Structure | Best For |
|----------|-----------|----------|
| **Schema-based** | `catalog.bronze.table`<br/>`catalog.silver.table`<br/>`catalog.gold.table` | Simpler environments, shared governance |
| **Catalog-based** | `bronze_catalog.schema.table`<br/>`silver_catalog.schema.table`<br/>`gold_catalog.schema.table` | Strict isolation, separate permissions |

### Tier Characteristics

| Tier | Scope | Typical Content |
|------|-------|-----------------|
| **Bronze** | Source-specific | Raw {SOURCE_PLATFORM} tables, CDC logs, landing zone |
| **Silver** | Source-specific | Cleansed dimensions and facts, conformed schema |
| **Gold** | Application-specific | Business aggregates, KPIs, ML feature tables, serving layers |

### Target Table Format

| Format | When to Use |
|--------|-------------|
| **Delta Lake** (preferred) | Default for all managed tables; full feature support including CDF, time travel, liquid clustering |
| **Iceberg** | Required for external engine interoperability (e.g., {SOURCE_PLATFORM}, Trino reading via Iceberg REST) |
| **Delta with UniForm** | Best of both: Delta internally, Iceberg metadata exposed for external access |

## Schema Migration Process

Schema migration follows a structured process from extraction through validation.

<br />
<div class="mermaid">
flowchart LR
    E["<b>Extract Source DDL</b><br/><i>{SOURCE_PLATFORM}</i>"]
    M["<b>Map Data Types</b><br/><i>Source → Delta Lake</i>"]
    B["<b>Apply Best Practices</b><br/><i>Partitioning, Clustering</i>"]
    G["<b>Generate Target DDL</b><br/><i>Databricks SQL</i>"]
    C["<b>Create in Unity Catalog</b><br/><i>Catalogs, Schemas, Tables</i>"]
    V["<b>Validate Schema</b><br/><i>Compare Source vs Target</i>"]
    E --> M --> B --> G --> C --> V
    style E fill:#e3f2fd,stroke:#1976d2
    style M fill:#e3f2fd,stroke:#1976d2
    style B fill:#fff3e0,stroke:#ff9800
    style G fill:#fff3e0,stroke:#ff9800
    style C fill:#e8f5e9,stroke:#4caf50
    style V fill:#e8f5e9,stroke:#4caf50
</div>
<script type="module"> import mermaid from "https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs"; mermaid.initialize({ startOnLoad: true, theme: "default" }); </script>

### Key Decisions

| Decision | Options | Recommendation |
|----------|---------|----------------|
| **Table format** | Delta, Iceberg, Parquet | Delta (default for managed tables) |
| **Catalog structure** | Match source or redesign | Match initially, refactor later |
| **Partitioning** | Same as source or optimize | Evaluate per table based on query patterns |
| **Clustering** | Z-ORDER or Liquid Clustering | Liquid Clustering for new tables |

## Extract Source DDL

The first step in schema migration is extracting table definitions from {SOURCE_PLATFORM}. Use information schema queries or platform-specific DDL export commands to capture column names, data types, constraints, and table properties.

<div class="code-block" data-language="sql">
-- Example: Extract table metadata from {SOURCE_PLATFORM}
-- REPLACEME: Add platform-specific information_schema query

SELECT 
    table_catalog,
    table_schema,
    table_name,
    column_name,
    ordinal_position,
    data_type,
    is_nullable,
    column_default
FROM information_schema.columns
WHERE table_schema = 'my_schema'
ORDER BY table_name, ordinal_position;
</div>

<link href="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/themes/prism.min.css" rel="stylesheet" />
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/prism.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/components/prism-sql.min.js"></script>

<script>
(function() {
    document.querySelectorAll('.code-block').forEach(function(block) {
        var lang = block.getAttribute('data-language') || 'sql';
        var code = block.textContent.trim();
        var id = 'code-' + Math.random().toString(36).substr(2, 9);
        
        block.innerHTML = 
            '<div style="position:relative;margin:16px 0;">' +
                '<button class="copy-btn" style="position:absolute;top:8px;right:8px;padding:4px 12px;font-size:12px;background:#ddd;color:#333;border:1px solid #ccc;border-radius:4px;cursor:pointer;z-index:10;">Copy</button>' +
                '<pre style="background:#f8f8f8;border-radius:8px;padding:16px;padding-top:40px;overflow-x:auto;margin:0;border:1px solid #e0e0e0;"><code id="' + id + '" class="language-' + lang + '" style="font-family:Consolas,Monaco,monospace;font-size:14px;"></code></pre>' +
            '</div>';
        
        var codeEl = document.getElementById(id);
        codeEl.textContent = code;
        Prism.highlightElement(codeEl);
        
        block.querySelector('.copy-btn').onclick = function() {
            var t = document.createElement('textarea');
            t.value = code;
            document.body.appendChild(t);
            t.select();
            document.execCommand('copy');
            document.body.removeChild(t);
            this.textContent = '✓ Copied!';
            setTimeout(() => this.textContent = 'Copy', 2000);
        };
    });
})();
</script>

## Data Type Mapping

Data types must be mapped carefully to avoid precision loss or runtime errors. The table below shows common mappings from {SOURCE_PLATFORM} to Databricks.

| <span style="white-space: nowrap;"><img src="https://cdn.simpleicons.org/snowflake/29B5E8" width="20" height="20" style="vertical-align: middle;"> {SOURCE_PLATFORM}</span> | <span style="white-space: nowrap;"><img src="https://cdn.simpleicons.org/databricks/FF3621" width="20" height="20" style="vertical-align: middle;"> Databricks</span> | Notes |
|-------------------|------------|-------|
| `NUMBER` / `NUMERIC` / `DECIMAL` | `DECIMAL(p,s)` | Specify precision and scale explicitly |
| `INT` / `INTEGER` | `INT` | Direct mapping |
| `BIGINT` | `BIGINT` | Direct mapping |
| `SMALLINT` | `SMALLINT` | Direct mapping |
| `FLOAT` / `FLOAT4` | `FLOAT` | Direct mapping |
| `DOUBLE` / `FLOAT8` | `DOUBLE` | Direct mapping |
| `VARCHAR(n)` | `STRING` | Databricks STRING is unbounded |
| `CHAR(n)` | `STRING` | No fixed-width char in Spark |
| `TEXT` | `STRING` | Direct mapping |
| `BINARY` / `VARBINARY` | `BINARY` | Direct mapping |
| `BOOLEAN` | `BOOLEAN` | Direct mapping |
| `DATE` | `DATE` | Direct mapping |
| `TIME` | `STRING` or `TIMESTAMP` | No native TIME type |
| `TIMESTAMP` | `TIMESTAMP` | Direct mapping |
| `TIMESTAMP_NTZ` | `TIMESTAMP_NTZ` | Supported in DBR 13.3+ |
| `VARIANT` / `JSON` | `STRING` or `STRUCT` | Parse with `from_json()` |
| `ARRAY` | `ARRAY<type>` | Element type required |
| `OBJECT` / `MAP` | `MAP<K,V>` or `STRUCT` | Structure must be defined |

## Apply Best Practices

Migration is an opportunity to optimize your schema design. Apply Lakehouse best practices for partitioning, clustering, and table properties to maximize query performance and minimize storage costs.

### Partitioning Strategy

Partitioning improves query performance by limiting data scans. Choose partition columns based on common filter predicates.

<br />
<div class="mermaid">
flowchart LR
    subgraph GOOD["<b>Good Partitioning</b>"]
        G1["Date columns<br/><i>event_date, created_date</i>"]
        G2["Low cardinality<br/><i>region, status</i>"]
        G3["Common filters<br/><i>frequently in WHERE</i>"]
    end
    subgraph BAD["<b>Avoid</b>"]
        B1["High cardinality<br/><i>user_id, transaction_id</i>"]
        B2["Many partitions<br/><i>> 10,000 partitions</i>"]
        B3["Small files<br/><i>< 100MB per partition</i>"]
    end
    style GOOD fill:#e8f5e9,stroke:#4caf50
    style BAD fill:#ffebee,stroke:#f44336
</div>
<script type="module"> import mermaid from "https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs"; mermaid.initialize({ startOnLoad: true, theme: "default" }); </script>

### Clustering Options

| Method | Use Case | Syntax |
|--------|----------|--------|
| **Liquid Clustering** | New tables, automatic optimization | `CLUSTER BY (col1, col2)` |
| **Z-ORDER** | Existing tables, manual optimization | `OPTIMIZE table ZORDER BY (col)` |

### Recommended Table Properties

<div class="code-block" data-language="sql">
-- Enable recommended table properties for new tables
ALTER TABLE catalog.schema.table_name SET TBLPROPERTIES (
    'delta.enableChangeDataFeed' = 'true',           -- Enable CDC for downstream
    'delta.autoOptimize.optimizeWrite' = 'true',     -- Automatic file sizing
    'delta.autoOptimize.autoCompact' = 'true',       -- Automatic compaction
    'delta.deletedFileRetentionDuration' = 'interval 7 days'  -- Time travel retention
);

-- Create table with Liquid Clustering
CREATE TABLE catalog.schema.my_table (
    id BIGINT,
    event_date DATE,
    customer_id BIGINT,
    amount DECIMAL(18,2),
    created_at TIMESTAMP
)
CLUSTER BY (event_date, customer_id)
TBLPROPERTIES (
    'delta.enableChangeDataFeed' = 'true'
);
</div>

<link href="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/themes/prism.min.css" rel="stylesheet" />
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/prism.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/components/prism-sql.min.js"></script>

<script>
(function() {
    document.querySelectorAll('.code-block').forEach(function(block) {
        var lang = block.getAttribute('data-language') || 'sql';
        var code = block.textContent.trim();
        var id = 'code-' + Math.random().toString(36).substr(2, 9);
        
        block.innerHTML = 
            '<div style="position:relative;margin:16px 0;">' +
                '<button class="copy-btn" style="position:absolute;top:8px;right:8px;padding:4px 12px;font-size:12px;background:#ddd;color:#333;border:1px solid #ccc;border-radius:4px;cursor:pointer;z-index:10;">Copy</button>' +
                '<pre style="background:#f8f8f8;border-radius:8px;padding:16px;padding-top:40px;overflow-x:auto;margin:0;border:1px solid #e0e0e0;"><code id="' + id + '" class="language-' + lang + '" style="font-family:Consolas,Monaco,monospace;font-size:14px;"></code></pre>' +
            '</div>';
        
        var codeEl = document.getElementById(id);
        codeEl.textContent = code;
        Prism.highlightElement(codeEl);
        
        block.querySelector('.copy-btn').onclick = function() {
            var t = document.createElement('textarea');
            t.value = code;
            document.body.appendChild(t);
            t.select();
            document.execCommand('copy');
            document.body.removeChild(t);
            this.textContent = '✓ Copied!';
            setTimeout(() => this.textContent = 'Copy', 2000);
        };
    });
})();
</script>

## Generate and Execute Target DDL

With data types mapped and best practices applied, generate Databricks-compatible CREATE TABLE statements. Execute them to create catalogs, schemas, and tables in Unity Catalog.

<div style="border-left: 4px solid #1976d2; background: #e3f2fd; padding: 16px 20px; border-radius: 4px; margin: 16px 0;">
    <div style="display: flex; align-items: flex-start; gap: 12px;">
        <span style="font-size: 24px;">ℹ️</span>
        <div>
            <strong style="color: #0d47a1; font-size: 1.1em;">Target State: Managed Tables</strong>
            <p style="margin: 8px 0 0 0; color: #333;">
                The goal should always be to end with <strong>managed tables</strong> in <strong>Delta Lake</strong> format. Managed tables provide full Unity Catalog governance, automatic file management, and access to optimization features like Predictive Optimization and Liquid Clustering.
            </p>
        </div>
    </div>
</div>

<div class="code-block" data-language="sql">
-- Create catalog and schema hierarchy
CREATE CATALOG IF NOT EXISTS prod_catalog;
USE CATALOG prod_catalog;

CREATE SCHEMA IF NOT EXISTS bronze;
CREATE SCHEMA IF NOT EXISTS silver;
CREATE SCHEMA IF NOT EXISTS gold;

-- Example converted table
CREATE TABLE IF NOT EXISTS prod_catalog.bronze.customers (
    customer_id BIGINT NOT NULL,
    first_name STRING,
    last_name STRING,
    email STRING,
    created_at TIMESTAMP,
    updated_at TIMESTAMP,
    _source_file STRING,
    _load_timestamp TIMESTAMP
)
CLUSTER BY (customer_id)
COMMENT 'Raw customer data from {SOURCE_PLATFORM}'
TBLPROPERTIES (
    'delta.enableChangeDataFeed' = 'true',
    'delta.autoOptimize.optimizeWrite' = 'true'
);
</div>

<link href="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/themes/prism.min.css" rel="stylesheet" />
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/prism.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/components/prism-sql.min.js"></script>

<script>
(function() {
    document.querySelectorAll('.code-block').forEach(function(block) {
        var lang = block.getAttribute('data-language') || 'sql';
        var code = block.textContent.trim();
        var id = 'code-' + Math.random().toString(36).substr(2, 9);
        
        block.innerHTML = 
            '<div style="position:relative;margin:16px 0;">' +
                '<button class="copy-btn" style="position:absolute;top:8px;right:8px;padding:4px 12px;font-size:12px;background:#ddd;color:#333;border:1px solid #ccc;border-radius:4px;cursor:pointer;z-index:10;">Copy</button>' +
                '<pre style="background:#f8f8f8;border-radius:8px;padding:16px;padding-top:40px;overflow-x:auto;margin:0;border:1px solid #e0e0e0;"><code id="' + id + '" class="language-' + lang + '" style="font-family:Consolas,Monaco,monospace;font-size:14px;"></code></pre>' +
            '</div>';
        
        var codeEl = document.getElementById(id);
        codeEl.textContent = code;
        Prism.highlightElement(codeEl);
        
        block.querySelector('.copy-btn').onclick = function() {
            var t = document.createElement('textarea');
            t.value = code;
            document.body.appendChild(t);
            t.select();
            document.execCommand('copy');
            document.body.removeChild(t);
            this.textContent = '✓ Copied!';
            setTimeout(() => this.textContent = 'Copy', 2000);
        };
    });
})();
</script>

## Schema Validation

Before loading data, verify that target schemas correctly reflect the source structure. Catch type mismatches, missing columns, and constraint issues early to avoid data load failures.

### Validation Checklist

| Check | Method | Expected Outcome |
|-------|--------|------------------|
| **Table count** | Compare source vs target catalogs | Exact match |
| **Column names** | Schema diff tool or query | All columns present |
| **Data types** | Type mapping review | Compatible types |
| **Nullable constraints** | Schema comparison | Constraints preserved |
| **Partitioning** | DESCRIBE DETAIL | Correct partition columns |
| **Table properties** | SHOW TBLPROPERTIES | Properties applied |

<div class="code-block" data-language="sql">
-- Validate table structure in Databricks
DESCRIBE TABLE EXTENDED catalog.schema.my_table;

-- Check table properties
SHOW TBLPROPERTIES catalog.schema.my_table;

-- Compare column counts between source and target
SELECT 
    table_name,
    COUNT(*) as column_count
FROM information_schema.columns
WHERE table_catalog = 'prod_catalog'
  AND table_schema = 'bronze'
GROUP BY table_name
ORDER BY table_name;
</div>

<link href="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/themes/prism.min.css" rel="stylesheet" />
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/prism.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/components/prism-sql.min.js"></script>

<script>
(function() {
    document.querySelectorAll('.code-block').forEach(function(block) {
        var lang = block.getAttribute('data-language') || 'sql';
        var code = block.textContent.trim();
        var id = 'code-' + Math.random().toString(36).substr(2, 9);
        
        block.innerHTML = 
            '<div style="position:relative;margin:16px 0;">' +
                '<button class="copy-btn" style="position:absolute;top:8px;right:8px;padding:4px 12px;font-size:12px;background:#ddd;color:#333;border:1px solid #ccc;border-radius:4px;cursor:pointer;z-index:10;">Copy</button>' +
                '<pre style="background:#f8f8f8;border-radius:8px;padding:16px;padding-top:40px;overflow-x:auto;margin:0;border:1px solid #e0e0e0;"><code id="' + id + '" class="language-' + lang + '" style="font-family:Consolas,Monaco,monospace;font-size:14px;"></code></pre>' +
            '</div>';
        
        var codeEl = document.getElementById(id);
        codeEl.textContent = code;
        Prism.highlightElement(codeEl);
        
        block.querySelector('.copy-btn').onclick = function() {
            var t = document.createElement('textarea');
            t.value = code;
            document.body.appendChild(t);
            t.select();
            document.execCommand('copy');
            document.body.removeChild(t);
            this.textContent = '✓ Copied!';
            setTimeout(() => this.textContent = 'Copy', 2000);
        };
    });
})();
</script>

## Summary

### Schema Migration Checklist

- [ ] Extract source DDL from {SOURCE_PLATFORM}
- [ ] Map all data types to Databricks equivalents
- [ ] Design Unity Catalog hierarchy (catalogs, schemas)
- [ ] Apply partitioning and clustering strategy
- [ ] Generate and execute target DDL
- [ ] Validate schema compatibility

### Key Principles

| Principle | Why It Matters |
|-----------|----------------|
| **Type safety** | Ensure no precision loss in type conversion, especially for DECIMAL/NUMBER |
| **Start simple** | Match source structure initially, refactor to medallion architecture later |
| **Optimize incrementally** | Add partitioning and clustering based on actual query patterns |
| **Automate** | Use Lakebridge or custom scripts for repeatability |

### Common Pitfalls

| Pitfall | Prevention |
|---------|------------|
| Precision loss | Use DECIMAL for financial data, verify scale/precision |
| Timezone issues | Document TZ handling; use TIMESTAMP_NTZ if local time needed |
| Over-partitioning | Limit to 1-2 partition columns; avoid high-cardinality columns |
| Missing constraints | Review NOT NULL constraints; document primary keys |

### Next Steps

With schemas created, proceed to data replication:

- [**3.3 - Data Migration and Ingestion**]($./3.3 - Data Migration and Ingestion) - Replication patterns from snapshot to CDC

<div style="color: #FF3621; font-weight: bold; font-size: 2em; margin-bottom: 12px;">COURSE DEVELOPER (remove before publishing)</div>

### Template Customization

**Placeholders to replace:**
- `{SOURCE_PLATFORM}` - Source platform name (Snowflake, BigQuery, Redshift, Teradata)

**Platform-specific additions required:**
- Add platform-specific type mapping table (e.g., Snowflake VARIANT, BigQuery STRUCT)
- Include platform-specific DDL extraction commands
- Add platform-specific information schema queries
- Document any platform-specific constraints or limitations

&copy; 2026 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>
